In [2]:

# Test: Filter Unwanted Files (Multi-Season)
# Verifies that result, standing, and round files are correctly excluded
# from Draft folders across multiple seasons in Google Drive.
#
# What it does:
#   1. Authenticates with Google Drive and locates the configured seasons
#   2. Scans each Draft folder in each season for files
#   3. Checks that the expected number of non-player files are excluded per folder
#      (R1, R2, R3, R4 + Standings/Results)
#   4. Reports any mismatches or files that slipped through the filter
#
# Supported draft folder name formats:
#   - YYYYMMDD Draft N
#   - Draft N YYYYMMDD

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import re

# Configuration
from config import SCOPES, MAIN_FOLDER_ID
SEASONS_TO_TEST = ['Season 4', 'Season 5']

# Expected number of excluded files per season (R1, R2, R3, R4, Standings/Results)
EXPECTED_EXCLUDED = {
    'Season 4': 5,
    'Season 5': 3,
}

# Matches both: "YYYYMMDD Draft N" and "Draft N YYYYMMDD"
DRAFT_FOLDER_PATTERN = r'(\d{8}\s+Draft\s+\d+|Draft\s+\d+\s+\d{8})'

print('Starting authentication...')

# Authenticate
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    print('✓ Using existing token.json')
else:
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
    print('✓ Created new token.json')

service = build('drive', 'v3', credentials=creds)
print('✓ Connected to Google Drive\n')

def get_folders(parent_id, name_pattern=None):
    """Get non-trashed folders from a parent folder."""
    query = f"'{parent_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    results = service.files().list(q=query, fields='files(id, name)').execute()
    folders = results.get('files', [])
    if name_pattern:
        folders = [f for f in folders if re.search(name_pattern, f['name'])]
    return folders

def get_files(parent_id):
    """Get non-trashed files from a folder, sorted by name."""
    query = f"'{parent_id}' in parents and trashed=false"
    results = service.files().list(q=query, fields='files(id, name)').execute()
    files = results.get('files', [])
    return sorted(files, key=lambda x: x['name'])

def is_player_file(filename):
    """Check if file is a player file (not a numbered round, results, or standings file)."""
    name_lower = filename.lower()
    if re.search(r'\d', filename):  # Any filename containing a number
        return False
    if 'result' in name_lower:
        return False
    if 'standing' in name_lower:
        return False
    return True

def extract_draft_sort_key(folder_name):
    """Extract a sortable date key from either draft folder format."""
    # Format 1: "YYYYMMDD Draft N" → leading date
    m = re.match(r'(\d{8})\s+Draft', folder_name)
    if m:
        return m.group(1)
    # Format 2: "Draft N YYYYMMDD" → trailing date
    m = re.search(r'Draft\s+\d+\s+(\d{8})', folder_name)
    if m:
        return m.group(1)
    return folder_name  # fallback: sort by name

# Find all season folders
print('Finding all season folders...')
all_season_folders = get_folders(MAIN_FOLDER_ID, r'Season \d+')
print(f'Found {len(all_season_folders)} season folders: {[f["name"] for f in all_season_folders]}\n')

# Global results
total_issues = False
all_failed = []

for season_name in SEASONS_TO_TEST:
    print(f'=' * 50)
    print(f'SEASON: {season_name}')
    print(f'=' * 50)

    expected = EXPECTED_EXCLUDED.get(season_name, 5)

    season = next((f for f in all_season_folders if f['name'] == season_name), None)
    if not season:
        print(f'❌ {season_name} not found! Skipping.\n')
        continue

    folders_in_season = get_folders(season['id'])
    pictures_folder = next((f for f in folders_in_season if f['name'].lower() == 'pictures'), None)
    if not pictures_folder:
        print(f'❌ Pictures folder not found in {season_name}! Skipping.\n')
        continue

    draft_folders = get_folders(pictures_folder['id'], DRAFT_FOLDER_PATTERN)
    if not draft_folders:
        print(f'❌ No Draft folders found in {season_name} > Pictures! Skipping.\n')
        continue

    draft_folders = sorted(draft_folders, key=lambda f: extract_draft_sort_key(f['name']))
    print(f'Found {len(draft_folders)} Draft folders (expecting {expected} excluded files each)\n')

    season_issues = False
    season_failed = []
    detailed_output = []

    for draft in draft_folders:
        output = []
        output.append(f'Checking: {draft["name"]}')

        all_files = get_files(draft['id'])
        excluded_files = [f for f in all_files if not is_player_file(f['name'])]
        player_files = [f for f in all_files if is_player_file(f['name'])]

        output.append(f'  Total files: {len(all_files)}')
        output.append(f'  Player files detected: {len(player_files)}')
        output.append(f'  Excluded files: {len(excluded_files)}')

        folder_has_issues = False

        if len(excluded_files) != expected:
            output.append(f'  🚨 ALERT: Expected {expected} excluded files, found {len(excluded_files)}!')
            season_issues = True
            total_issues = True
            folder_has_issues = True

        if excluded_files:
            output.append(f'  ✓ Correctly excluded:')
            for f in excluded_files:
                output.append(f'    - {f["name"]}')
        else:
            output.append(f'  ⚠️  No files were excluded!')

        for f in player_files:
            name_lower = f['name'].lower()
            if re.search(r'\d', f['name']) or 'result' in name_lower or 'standing' in name_lower:
                output.append(f'  ⚠️  WARNING: This file should probably be excluded: {f["name"]}')
                season_issues = True
                total_issues = True
                folder_has_issues = True

        if folder_has_issues:
            season_failed.append({
                'season': season_name,
                'name': draft['name'],
                'excluded_count': len(excluded_files),
                'excluded_files': [f['name'] for f in excluded_files]
            })

        detailed_output.append('\n'.join(output))

    # Season summary
    print(f'TEST RESULTS for {season_name}\n')
    if not season_issues:
        print(f'✅ All checks passed for {season_name}!\n')
    else:
        print(f'Issues found in {season_name}:\n')
        for folder in season_failed:
            print(f'  📁 {folder["name"]}')
            print(f'     - Excluded files: {folder["excluded_count"]} (expected {expected})')
            if folder['excluded_files']:
                print(f'     - Files excluded: {", ".join(folder["excluded_files"])}')
            else:
                print(f'     - Files excluded: None')
            print()

    print(f'DETAILED CHECK RESULTS for {season_name}\n')
    for output in detailed_output:
        print(output)
        print()

    all_failed.extend(season_failed)

# Overall summary
print('=' * 50)
print('OVERALL SUMMARY')
print('=' * 50)
if not total_issues:
    print(f'✅ All checks passed across all {len(SEASONS_TO_TEST)} seasons!')

assert not total_issues, (
    f'{len(all_failed)} folder(s) with issues across {len(SEASONS_TO_TEST)} seasons:\n' +
    '\n'.join(
        f'  - [{f["season"]}] {f["name"]}: {f["excluded_count"]} excluded '
        f'(expected {EXPECTED_EXCLUDED.get(f["season"], 5)})'
        for f in all_failed
    )
)


Starting authentication...
✓ Using existing token.json
✓ Connected to Google Drive

Finding all season folders...
Found 5 season folders: ['Season 5', 'Season 4', 'Season 3', 'Season 2', 'Season 1']

SEASON: Season 4
Found 12 Draft folders (expecting 5 excluded files each)

TEST RESULTS for Season 4

✅ All checks passed for Season 4!

DETAILED CHECK RESULTS for Season 4

Checking: Draft 1 20240922
  Total files: 17
  Player files detected: 12
  Excluded files: 5
  ✓ Correctly excluded:
    - D1R1.jpeg
    - D1R2.jpeg
    - D1R3.jpeg
    - D1R4.jpeg
    - Results.jpg

Checking: Draft 2 20241020
  Total files: 17
  Player files detected: 12
  Excluded files: 5
  ✓ Correctly excluded:
    - R1.jpeg
    - R2.jpeg
    - R3.jpeg
    - R4.jpeg
    - Standings.jpeg

Checking: Draft 3 20241124
  Total files: 17
  Player files detected: 12
  Excluded files: 5
  ✓ Correctly excluded:
    - D3 Standing.jpeg
    - R1.jpeg
    - R2.jpeg
    - R3.jpeg
    - R4.jpeg

Checking: Draft 4 20241208
  Total